In [1]:
import os

input_file = "../media/sample1.mp3"

In [2]:
# Prepare output audio file
audio_file = ""

# If video, extract audio
if input_file.endswith((".mp4", ".mov", ".mkv", ".avi")):
    os.system(f"ffmpeg -i \"{input_file}\" -ar 16000 -ac 1 -c:a pcm_s16le {audio_file}")
else:
    audio_file = input_file  # Already an audio file

In [3]:
import transformers
print(transformers.__version__) 

c:\Users\adabh\Documents\projects\myproject\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.52.3


In [4]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch
import torchaudio

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("kiranpantha/whisper-large-v3-nepali").to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


In [5]:
import ffmpeg
# Convert audio file to WAV format with 16kHz sample rate and mono channel
def convert_audio_to_wav(input_file, output_file):
    (
        ffmpeg
        .input(input_file)
        .output(output_file, ar=16000, ac=1, format='wav')
        .run(overwrite_output=True)
    )
# Convert the audio file to WAV format
wav_file = "../media/converted_audio.wav"
convert_audio_to_wav(audio_file, wav_file)

In [6]:
import librosa

# Load audio using librosa
audio, sr = librosa.load("../media/converted_audio.wav", sr=16000)
print(f"Audio shape: {audio.shape}, Sample rate: {sr}")

Audio shape: (103680,), Sample rate: 16000


In [7]:
inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to(device)

In [9]:
# Transcribe (file path works here directly)
with torch.no_grad():
    generated_ids = model.generate(**inputs,attention_mask = inputs["attention_mask"].to(device) if "attention_mask" in inputs else None)

# Decode to text
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Transcription:", transcription)

Transcription: बुधबहर कप्ताको मेरो मनपर्ने दिन हो।


In [ ]:

reference = "बुधवार हप्ताको मेरो मनपर्ने दिन हो । "
import jiwer

# Define normalization pipeline
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemovePunctuation(),
    jiwer.Strip(),
    jiwer.RemoveMultipleSpaces(),
])

# Apply transformation and calculate WER
normalized_reference = transformation(reference)
normalized_hypothesis = transformation(transcription)

error = jiwer.wer(normalized_reference, normalized_hypothesis)

print(f"Normalized WER: {error:.3f} ({error*100:.2f}%)")

बुधवार हप्ताको मेरो मनपर्ने दिन हो
Normalized WER: 0.333 (33.33%)
